In [13]:
import pandas as pd
df = pd.read_csv('/content/sample_data/03_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [14]:
%pip install catboost

In [15]:
%pip install lightbgm

ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostRegressor as Catboost
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

stacking_reg = StackingRegressor(
    estimators=[
        ('lin_reg', LinearRegression()),
        ('LGBM', LGBMRegressor(random_state=42)),
        ('xgboost_01', xgb.XGBRegressor(random_state=42)),
        ('xgboost_02', xgb.XGBRegressor(random_state=42)),
        ('catboost_01', Catboost(learning_rate=1, random_state=42)),
        ('catboost_02', Catboost(learning_rate=1, random_state=42)),
    ],
    final_estimator=LinearRegression(),
    n_jobs=-1,
    cv=3
)

param_distribs = {
    'LGBM__n_estimators': randint(low=50, high=120),
    'xgboost_01__max_depth': randint(low=6, high=12),
    'xgboost_01__n_estimators': randint(low=30, high=80),
    'xgboost_02__max_depth': randint(low=6, high=12),
    'xgboost_02__n_estimators': randint(low=30, high=80),
    'catboost_01__iterations': randint(low=6, high=12),
    'catboost_01__depth': randint(low=5, high=15),
    'catboost_02__iterations': randint(low=6, high=12),
    'catboost_02__depth': randint(low=5, high=15),
}

param_search = RandomizedSearchCV(stacking_reg, param_distributions=param_distribs, n_iter=5, random_state=42)
param_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/loc

In [ ]:
from sklearn.metrics import root_mean_squared_error
y_pred = stacking_reg.predict(X_train)
rmsle = root_mean_squared_error(y_train, y_pred)
rmsle

In [ ]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()

In [ ]:
y_pred = stacking_reg.predict(X_test)
rmsle = root_mean_squared_error(y_test, y_pred)
rmsle

In [ ]:
# save final prediction
import numpy as np

test_set = pd.read_csv('/content/sample_data/03_test.csv')
del test_set['Premium Amount']
y_pred = stacking_reg.predict(test_set)
y_pred = np.exp(y_pred)
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('stacking_reg_01.csv', index=False)